# Early optimization trials

## Background

Recall that the goal was to optimize the cooling capacity of a heat-driven cooling system, by adjusting the internal design parameters of the chiller subsystem and heat exchangers. In optimization language, the objective to minimize is the additive inverse of the cooling capacity. Three external streams are treated as inputs defined at their inlet to the heat exchangers (one as heat supply, one for heat rejection, and another to deliver cooling to the load). Furthermore, the goal for the maximum "cost" (monetary, material, etc) associated with the design is treated as a constraint. The challenges with delivering satisfactory optimization trials appear to come from properly constraining the optimization problem, and possibly from insufficient smoothness due to numerical methods for evaluating heat transfer.

The internal design parameters are the sizes of the heat exchangers, pumped mass flow rates, and valve coefficients if present. However, with heat exchanger sizes as the inputs, solving requires iteration to determine fluid states at key points (similar to a transient solver converging to steady state). Despite potential disadvantages, we wanted to go with the simplest possible model, so instead I treated as inputs the fluid states at key points (a handful of temperatures and absorbate concentrations), which decouples solving the internal cycle from the external streams. Then the model is solved in one pass. The downside is that it becomes possible to give the model an infeasible set of inputs, for which the cycle doesn't solve but yields an error. Constraints that make the cooling cycle feasible are called **hard constraints** (they must be always observed). Constraints that pertain to feasible heat transfer and cost goal are considered **soft constraints**, because they can be violated during the process of optimization as long as they eventually are resolved. Going forward, we could change out some hard constraints by rearranging the model to group blocks of the cycle that could be solved independently, resulting in mismatch in mass/species/energy instead of a solver error, and treat the mismatch as a soft constraint (more like the equation-solver approach).

Originally, I developed my models intending to use the [optimization library from Scipy](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html), called directly in Python from module `scipy.optimize`. After early trials failed to yield satisfying results, I attempted to use [GenOpt tool from LBL](https://simulationresearch.lbl.gov/GO/) (a standalone Generic Optimization program coded in Java, with [source on GitHub](https://github.com/lbl-srg/GenOpt), not to be confused with any other GenOpt). Below I will show the results of the early (ie. not yet satisfying) trials for the single-effect ammonia-water absorption cycle system.

To clarify language, the code uses classes to encapsulate the following objects seen in source files `ammonia1.py` and `system_aqua1.py`:

- `Problem(System sys, goal_UA)`. The optimization problem currently being solved including methods to evaluate the objective, constraints, solver options. This object calls on a System object to handle model calculations.
- `System(Boundary bdry, Chiller ch)`. Couples together calculations involving the cooling subsystem and the external streams.
- `Chiller(chiller spec)`. The model for the chiller subsystem defined by a specification; for the set of input variables, a shortcut is provided as `makeChiller(xC)` for the input vector `xC = mdot, T1, T2, ...`.
- `Boundary(Stream heat_supply, Stream heat_rejection, Stream cooling)`. The set of external streams. For convenient initialization and hashing, a shortcut is provided as `makeBoundary(xB)` where the input vector `xB = (T_inlet_1, m_1, T_inlet_2, m_2, ...)`. (Although the shortcut generates three sensible streams with the specific heat of liquid water, this does not mean that you have designed a water-cooled system; you could change $c_p$ and expect the same results, as discussed in the prospectus. A future revision should clarify by asking to input $\dot m c_p$ instead of $\dot m$, but meanwhile I already have the trials indexed by this definition of `xB`.)
- `Stream`. Must implement methods `q(T)` and `T(q)` to describe the (q,T) curve for a particular process. For example, a straight-through flow process would have $q = \dot m \Delta h$. A desorber or absorber may have internally counterflowing gas and liquid streams, so the equation takes on a different form (confer prior writings and textbooks).

We set up the problem in order to look at parameter studies where we loop over a range of values for some variable fixed during optimization, such as `goal_UA` or `xB`, so these are described by vectors in order to hash and store the results for easy lookup.

### Handling constraints

In the scipy library there are some bounded and constrainted optimization solvers, allowing for constraints to be passed in as a vector separately from the objective. That is easy, so in my first attempt, the system includes a list of all constraints in the system object.

However, other than box constraints (variable bounds), GenOpt directs the user to build a custom Optimizer or incorporate the constraints into a modified objective function. The GenOpt manual includes a discussion of numerical methods, such as barrier and penalty functions, and these methods are adopted when optimizing with GenOpt.

Note that scipy.optimize takes the convention that inequality constraints are of the form

$$ C_j(x) \ge 0 $$

whereas GenOpt takes the convention that inequality constraints are of the form

$$ g_i(x) \le 0, $$

so the code has to wrap the outputs depending on the choice of optimizer interface.

### Handling heat transfer

In the `HRHX_integral_model` (since [1e79c81](https://github.com/nfette/openACHP/commit/1e79c81e4ea44572713f2f755ef70c20d6b8732c#diff-657bfcb686553b7168f7119358de9898)), I use the integral version of the counterflow heat exchange equation relating UA and Q (see my prospectus, eqn. #10), which depends on two curves, (Q, T) for the cold and hot streams. With the current choice of inputs, the Q is fixed by the cooling cycle, we just need to perform the integral to get UA. The [integration library from Scipy](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html) provides numerical integration routines, so I used that. Naturally, this results in long computation times to evaluate UA given Q. (Furthermore, if effectiveness or pinch point are desired, that requires an additional call to a minimize routine. Sometimes I have tried to use as constraints the effectiveness or $\Delta T$ at pinch.) This is because for each stream, evaluating each point on the (q,T) curve involves fluid property lookups, including possibly iterative solves with enthalpy as input, and a little arithmetic. Thus in commit [1726931](https://github.com/nfette/openACHP/commit/1726931267697a4abbed7557a59d36680f97fbf8#diff-657bfcb686553b7168f7119358de9898), I introduced a trick intended to reduce computation time: to limit the number of property lookups, I make a spline of the (q,T) curve prior to integration (with a new stream object). Per point, the spline interpolate is faster than the direct property lookup, and the inverse curve is obtained for free. I have chosen to build the spline from a fixed number of points, which introduces potential error from discretization. TODO: profile the code to produce a table comparing the time to evaluate a System model with and without the interpolate.

At the moment, to evaluate Q given UA, I just minimize the error in UA, because I already have a function UA(Q). This is not ideal, because Q(UA) is the more physical problem (always feasible to solve). It would make sense to use a finite element method. Perhaps that would be a good future addition.

## Code samples

### Input vectors

Here is how we map the Boundary and Chiller specifications. From `system_aqua1.py`:

```python
def makeBoundary(x):
    """Given a iterable x that represents 5 stream inlets as T,m pairs,
    returns the Boundary object representing that.
    
    t0,m0,t1,m1,t2,m2,t3,m3,t4,m4 = x
    
    Note that boundary streams are
    
    0
        heat
    1
        absorberReject
    2
        condenserReject
    3
        cold
    4
        rectifierReject
    """
    t0, m0, t1, m1, t2, m2, t3, m3, t4, m4 = x
    # Units: K, kg/s, kW/kg-K
    cp = 4.179
    return Boundary(
        heat=streamExample1(t0, m0, cp),
        absorberReject=streamExample1(t1, m1, cp),
        condReject=streamExample1(t2, m2, cp),
        cold=streamExample1(t3, m3, cp),
        rectifierReject=streamExample1(t4, m4, cp))


def makeChiller(x):
    """Given a iterable x that represents a chiller state,
    return a chiller object.
    
    Args
    ====
    
    x[0]
        m_rich (kg/s)
    x[1]
        T_evap (K)
    x[2]
        T_cond (K)
    x[3]
        T_rect (K)
    x[4]
        T_abs_outlet (K)
    x[5]    
        T_gen_outlet (K)
    """
    chiller = ammonia1.AmmoniaChiller()
    m_rich, T_evap, T_cond, T_rect, T_abs_outlet, T_gen_outlet = x
    chiller.update(m_rich=m_rich,
                   T_evap=T_evap,
                   T_cond=T_cond,
                   T_rect=T_rect,
                   T_abs_outlet=T_abs_outlet,
                   T_gen_outlet=T_gen_outlet)
    return chiller
```

### Trials with scipy.optimize, constrained optimization
Here is how we computed constraints for constrained optimization solvers in scipy.optimize. The constraint functions (desired to be non-negative) passed to the optimizer are:

\begin{align}
C_0 &= m_{rich} \\
C_1 &= T_{cond} - T_{evap} \\
C_2 &= T_{rect} - T_{cond} \\
C_3 &= T_{generator\ outlet} - T_{rect} \\
C_4 &= T_{generator\ outlet} - T_{absorber\ outlet} \\
C_5 &= \Delta T_{generator} \\
C_6 &= \Delta T_{rectifier} \\
C_7 &= \Delta T_{absorber} \\
C_8 &= \Delta T_{condenser} \\
C_{9} &= \Delta T_{evaporator} \\
C_{10} &= UA_{goal} - UA_{actual}
\end{align}

The code that accomplishes this includes a lookup function so that the System() is only called once per unique combination of inputs, but otherwise is equivalent to:

```python
class Problem(object):
    
    def __init__(self, bdry, UAgoal, constraintMode=None, mu=0.1):
        self.bdry = bdry
        self.UAgoal = UAgoal
        self.constraintMode = constraintMode
        self.mu = 0.1
        self.input = []
        self.output = dict()
        self.Ncons = 11
        if constraintMode == None:
            # Automatic constraints mode: minimizer imposes all constraints.
            # This object is sent to minimizer with handles back to self.
            # See definition of constraint() below.
            # Confusingly, some code comments called this soft constraints mode.
            self.constraints = [{'type': 'ineq',
                                 'fun': self.constraint,
                                 'args': (i,)} for i in range(self.Ncons)]
        else:
            # Manual constraints modes: objective function modified.
            # See definition of objective() below.
            # Confusingly, some code comments called this hard constraints mode.
            self.constraints = []

    def objective(self, x, stepNumber=1):
        """Return the value of the objective function to minimize. This is
        :math:`-Q_{evap}` modified depending on constraintMode.
        """
        try:
            sys = System(self.bdry, makeChiller(x))
            Q = sys.chiller.Q_evap
        except Exception as e:
            Q = 0
            
        if self.constraintMode != None:
            raise NotImplementedError
            
        return -Q

    def constraint(self, x, *args):
        i, = args
        cons = [x[0],
                x[2] - x[1],
                x[3] - x[2],
                x[5] - x[3],
                x[5] - x[4]]
        try:
            sys = System(self.bdry, makeChiller(x))
            # This loops over external-facing heat exchangers
            for name, deltaT, epsilon, UA, Qhx in sys.data:
                cons.append(deltaT)
            cons.append(self.UAgoal - sys.totalUA)
        except Exception as e:
            while len(cons) < self.Ncons:
                cons.append(-1)
        return cons[i]
            
def makeOrGetProblemForBoundary(
        xB, U, xC, method=None, options=None, folder='data', create=False):
    ...
    # Create the data
    opt, err = None, None
    bdry = makeBoundary(xB)
    p = Problem(bdry, U, constraintMode=None)
    try:
        opt = minimize(p.objective,
                       xC,
                       constraints=p.constraints,
                       method=method,
                       options=options)
    ...
    return xB, bdry, p, opt, err

```

### Primitive barrier: first attempt

Here is how we compute the modified objective function. In the GenOpt trial 1 (`opt1` folder), I used my primitive barrier function, thus, in `src/system_aqua1.py`. Note that the loop `for row in sys.data` iterates over the external facing heat exchangers.

```python
def main2():
    """Script to run with GenOpt, using a stupid barrier function"""

    import sys  # command args
    import traceback  # display errors into log
    import json  # read input file with standard, structured format
    from decimal import Decimal  # convert 'inf' to 'Infinity' etc

    infile, outfile, logfile = sys.argv[1:4]

    with open(logfile, 'w') as log:
        print('infile="{}"'.format(infile), file=log)
        print('outfile="{}"'.format(outfile), file=log)
        print('logfile="{}"'.format(logfile), file=log)

        try:
            with open(infile) as fin:
                data = json.load(fin)
            stepNumber = data['stepNumber']
            UAgoal = data['UAgoal']
            xB = data['xB']
            xC = data['xC']
            print('UAgoal={}'.format(UAgoal), file=log)
            print('xB={}'.format(xB), file=log)
            print('xC={}'.format(xC), file=log)
            bdry = makeBoundary(xB)
            chiller = makeChiller(xC)
            sys = System(bdry, chiller)
            print(sys, file=log)

            with open(outfile, 'w') as fout:
                print("Q_evap=", chiller.Q_evap, file=fout)
                sum_g_barrier = 0
                for row in sys.data:
                    name, deltaT, epsilon, UA, Qhx = row
                    print("UA_{}={}".format(name, Decimal(UA)), file=fout)
                    print("deltaT_{}={}".format(name, Decimal(deltaT)), file=fout)
                    sum_g_barrier += deltaT
                print("sum_g_barrier={}".format(Decimal(sum_g_barrier)), file=fout)
                mu = 3 / stepNumber
                print("mu={}".format(mu), file=fout)

        except:
            traceback.print_exc(file=log)
```

I pass the value to GenOpt via the configuration file, `opt1\pythonSimulation.cfg`:

```json
ObjectiveFunctionLocation{
    Name1      = objective;
    Function1  = "add(-%Q_evap%, multiply(%mu%,-%sum_g_barrier%))";
    Name2      = Q_evap;
    Delimiter2 = "Q_evap=";
    Name3      = UA_gen; // gen rect abs cond evap
    Delimiter3 = "UA_gen=";
    Name4      = UA_rect;
    Delimiter4 = "UA_rect=";
    Name5      = UA_abs;
    Delimiter5 = "UA_abs=";
    Name6      = UA_cond;
    Delimiter6 = "UA_cond=";
    Name7      = UA_evap;
    Delimiter7 = "UA_evap=";
    Name8      = deltaT_gen; // gen rect abs cond evap
    Delimiter8 = "deltaT_gen=";
    Name9      = deltaT_rect;
    Delimiter9 = "deltaT_rect=";
    Name10      = deltaT_abs;
    Delimiter10 = "deltaT_abs=";
    Name11      = deltaT_cond;
    Delimiter11 = "deltaT_cond=";
    Name12      = deltaT_evap;
    Delimiter12 = "deltaT_evap=";
    Name13      = con1;
    Function13  = "%m_rich%";
    Name14      = con2;
    Function14  = "add(%T_cond%, -%T_evap%)";
    Name15      = con3;
    Function15  = "subtract(%T_rect%, %T_cond%)";
    Name16      = con4;
    Function16  = "subtract(%T_gen_outlet%, %T_rect%)";
    Name17      = con5;
    Function17  = "subtract(%T_gen_outlet%, %T_abs_outlet%)";
    Name18      = mu;
    Delimiter18  = "mu=";
    Name19      = sum_g_barrier;
    Delimiter19 = "sum_g_barrier=";
}
```

### Primitive barrier: second attempt

After the linear barrier function above, I tried applying the expit function (logistic curve) to smooth the barriers. This was a bad idea.

$$ Objective = -Q \prod_{j} \mathrm{expit}(C_j / \mu) $$

This was implemented in `Problem.objective()` thus:

```python
def objective(self, x, stepNumber=1):
    Q, cons = self.lookup(x, printing=True)
    if self.constraintMode == 'expit':
        mu = self.mu
        for c in cons:
            Q *= expit(c / mu)
        return -Q
    else:
        return -Q
```

### Constraint mode 'genopt1'

I added more sophisticated, separate barrier (B) and penalty (P) functions and passed these to the optimizer.
In GenOpt trial folders `opt2` through `opt5`, I used this technique. I may also have tried this with scipy.optimize (which trials?). The objective function is, (as documented in source),

$$ Objective = -Q(x) + \mu_1 B(x) + \mu_2 P(x) $$
$$ B(x) = \sum_{i=0}^{9} g_i$$
$$ \mu_1 = ...$$
$$ P(x) = \sum_{i=10}^{10} g_i$$
$$ \mu_2 = ...$$


This was implemented thus: for the scipy.optimize routines, in Python,

```python
def objective(self, x, stepNumber=1):
    Q, cons = self.lookup(x, printing=True)
    if self.constraintMode == 'genopt1':
        # Each cons[i] > 0 by scipy convention,
        # So need to change the sign of the second term.

        # GenOpt eqn. 8.6
        barrier = 0
        for c in cons[:10]:
            g = -c
            barrier += g
        # Eqn. 8.7
        mu1 = stepNumber ** (-2)

        # GenOpt eqn. 8.8
        penalty = 0
        for c in cons[10:]:
            g = -c
            penalty += max(0, g) ** 2
        # Eqn 8.9
        mu2 = stepNumber ** (2)

        f_tilde = -Q + mu1 * barrier + mu2 * penalty
        return f_tilde
    else:
        return -Q
```

And for the GenOpt optimizer, in Python:

```python
def systemConstraints(sys, UAgoal):
    T_evap = sys.chiller.refrig_evap_outlet.T
    T_cond = sys.chiller.refrig_cond_outlet.T
    T_rect = sys.chiller.refrig_rect_outlet.T
    T_gen_outlet = sys.chiller.weak_gen_outlet.T
    T_abs_outlet = sys.chiller.rich_abs_outlet.T
    hardConstraints = [T_cond - T_evap,
                       T_rect - T_cond,
                       T_gen_outlet - T_rect,
                       T_gen_outlet - T_abs_outlet]
    # name, deltaT, epsilon, UA, Q
    deltaTT = [hx[1] for hx in sys.data]
    for deltaT in deltaTT:
        hardConstraints.append(deltaT)

    softConstraints = [UAgoal - sys.totalUA]

    # GenOpt eqn. 8.6
    barrier = 0
    for c in hardConstraints:
        g = -c
        barrier += g
    # Eqn. 8.7
    # mu1 = stepNumber ** (-2)

    # GenOpt eqn. 8.8
    penalty = 0
    for c in softConstraints:
        g = -c
        penalty += max(0, g) ** 2
    # Eqn 8.9
    # mu2 = stepNumber ** (2)

    return barrier, penalty
    
    
def main3(infile, outfile, logfile):
    """Script to run with GenOpt, using more intelligent constraints."""

    import traceback  # display errors into log
    import json  # read input file with standard, structured format
    from decimal import Decimal  # convert 'inf' to 'Infinity' etc

    with open(logfile, 'w') as log:
        print('infile="{}"'.format(infile), file=log)
        print('outfile="{}"'.format(outfile), file=log)
        print('logfile="{}"'.format(logfile), file=log)

        try:
            with open(infile) as fin:
                data = json.load(fin)
            stepNumber = data['stepNumber']
            UAgoal = data['UAgoal']
            xB = data['xB']
            xC = data['xC']
            print('UAgoal={}'.format(UAgoal), file=log)
            print('xB={}'.format(xB), file=log)
            print('xC={}'.format(xC), file=log)
            bdry = makeBoundary(xB)
            chiller = makeChiller(xC)
            sys = System(bdry, chiller)
            print(sys, file=log)
            barrier, penalty = systemConstraints(sys, UAgoal) # Look here

            with open(outfile, 'w') as fout:
                if np.isinf(barrier) or np.isinf(penalty):
                    print("Q_evap=", -1, file=fout)
                    print("barrier=", 0, file=fout)
                    print("penalty=", 0, file=fout)
                else:
                    print("Q_evap=", chiller.Q_evap, file=fout)
                    print("barrier=", barrier, file=fout)
                    print("penalty=", penalty, file=fout)

        except:
            traceback.print_exc(file=log)

```

Then in the GenOpt configuration,

```json
ObjectiveFunctionLocation{
    Name1      = objective;
    Function1  = "add(multiply(-1,%Q_evap%), %barrier_and_penalty%)";
    Name2      = Q_evap;
    Delimiter2 = "Q_evap=";
    Name3      = barrier_and_penalty;
    Function3  = "add(multiply(%mu1%,%barrier%),multiply(%mu2%,%penalty%))";
    Name4      = barrier;
    Delimiter4 = "barrier=";
    Name5      = penalty;
    Delimiter5 = "penalty=";
    Name6      = mu1;
    Function6  = "pow(%stepNumber%,-2)";
    Name7      = mu2;
    Function7  = "pow(%stepNumber%,2)";
}
```


Some explanation precedes code.


## Trials with scipy.optimize